In [1]:
import whisper, time
import numpy as np
import torch

In [2]:
# Persian song
audio_file = "MohsenYeganeh.mp3"

In [3]:
model_number = 5

In [4]:
available_models = {
    1:'tiny',
    2:'base',
    3:'small',
    4:'medium',
    5:'large-v2'}

In [5]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device("cuda") if \
    torch.cuda.is_available() else torch.device("cpu")
print('Using device:', device)

Using device: cuda


In [6]:
whisper.available_models()

['tiny.en',
 'tiny',
 'base.en',
 'base',
 'small.en',
 'small',
 'medium.en',
 'medium',
 'large-v1',
 'large-v2',
 'large']

In [7]:
model = whisper.load_model(available_models[model_number]).to(device)

In [8]:
st = time.time()


result = model.transcribe(audio=audio_file, language='Persian', temperature=0.01, best_of=3)


ed = time.time()
print(f'time: {(ed-st)//60} min, {np.round((ed-st)%60)} sec')

time: 0.0 min, 19.0 sec


In [9]:
result.keys()

dict_keys(['text', 'segments', 'language'])

In [10]:
result['language']

'Persian'

In [11]:
result['text']

' PYM JBZ خودت می خواهید بری خاطرشی اما دلت می سوزه تذاکر میکنی عاشقمی این بازی هر روزه نترس آدم دمه رفتن همش دلشوره می گیره دو روز بگذرین دلشوره ها بهت قول میدم صفت نیز لاغال برای تو راحت کاش دورم از تو و دنیای تو راحت باش هیشگس نمیاد جای تو دلشوره دارم من بهت قول میدم صفت نیز لاغال برای تو راحت باش دورم از تو و دنیای تو راحت باش هیشگس نمیاد جای تو دلشوره دارم من منم از تو از عشق هرچیزی که میشناسم و از من گرفتی تو تو باقی منده احساسم و از من گرفتی و میخوای من باشی و یادت بره مایی وجود داره خودت آماده رفتنی هم ترست نمی ساره اصلا نترس راحت برو بی من هیشگی به جسد من را یادش نیز فکر که کی از من خبر داره راحت برو هیشگی حواستش نیز بهت بهت قول میدم صفت نیز لاغال برای تو راحت باش دورم از تو و دنیای تو راحت باش هیشگس نمیاد جای تو دلشوره دارم من بهت قول میدم صفت نیز لاغال برای تو راحت باش دورم از تو و دنیای تو راحت باش هیشگس نمیاد جای تو دلشوره دارم من موسیقی موسیقی موسیقی موسیقی'

In [12]:
result['segments']

[{'id': 0,
  'seek': 0,
  'start': 0.0,
  'end': 5.0,
  'text': ' PYM JBZ',
  'tokens': [50364, 430, 56, 44, 43019, 57, 50614],
  'temperature': 0.01,
  'avg_logprob': -0.31973081827163696,
  'compression_ratio': 0.4666666666666667,
  'no_speech_prob': 0.5177953839302063},
 {'id': 1,
  'seek': 3000,
  'start': 30.0,
  'end': 34.0,
  'text': ' خودت می خواهید بری خاطرشی',
  'tokens': [50364,
   16490,
   23328,
   2655,
   48478,
   16490,
   14407,
   3224,
   4135,
   3215,
   4724,
   44823,
   16490,
   41193,
   2288,
   8592,
   4135,
   50564],
  'temperature': 0.01,
  'avg_logprob': -0.29991918535374884,
  'compression_ratio': 1.7248677248677249,
  'no_speech_prob': 0.8658968806266785},
 {'id': 2,
  'seek': 3000,
  'start': 34.0,
  'end': 38.0,
  'text': ' اما دلت می سوزه تذاکر میکنی',
  'tokens': [50564,
   1975,
   15042,
   11778,
   1211,
   2655,
   48478,
   8608,
   2407,
   11622,
   3224,
   6055,
   15730,
   6007,
   2288,
   3714,
   29325,
   1863,
   4135,
   50764]

### Another example from Open-AI

In [13]:
model = whisper.load_model("large-v2")

# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio(audio_file)

# skipped the first 30 seconds to detect language correctly
temp = whisper.pad_or_trim(audio)         
audio = whisper.pad_or_trim(audio[len(temp):])


# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)

Detected language: fa
خودت می خواهی بری خاطرشی اما دلت می سوزه تظاهر میکنی عاشقمی این بازی هر روزه نترس آدم دمه رفتن همش دلشوره می گیره دو روز بگذرین دلشوره ها از قاعده از بین بهت قول میدم صفت نیز لغم برای تو راحت کاش


In [14]:
options = whisper.DecodingOptions(task='transcribe', # translate
                                  language= 'Persian',
                                  temperature= 0.3,
                                  best_of= 5)

In [15]:
options

DecodingOptions(task='transcribe', language='Persian', temperature=0.3, sample_len=None, best_of=5, beam_size=None, patience=None, length_penalty=None, prompt=None, prefix=None, suppress_tokens='-1', suppress_blank=True, without_timestamps=False, max_initial_timestamp=1.0, fp16=True)

### Command line, better quality in Persian

In [1]:
# translate transcribe
!whisper MohsenYeganeh.mp3 --language Persian --task transcribe  --model large-v2 \
                           --temperature 0.01 --output_dir transcribe --best_of 3

[00:00.000 --> 00:05.000]  PYM JBZ
[00:30.000 --> 00:34.000]  خودت می خواهید بری خاطرشی
[00:34.000 --> 00:38.000]  اما دلت می سوزه تذاکر میکنی
[00:38.000 --> 00:42.000]  عاشقمی این بازی هر روزه نترس
[00:42.000 --> 00:46.000]  آدم دمه رفتن همش دلشوره می گیره
[00:46.000 --> 00:50.000]  دو روز بگذرین دلشوره ها
[00:50.000 --> 00:55.000]  بهت قول میدم صفت نیز
[00:55.000 --> 00:59.000]  لاغال برای تو راحت کاش
[00:59.000 --> 01:04.000]  دورم از تو و دنیای تو راحت باش
[01:04.000 --> 01:11.000]  هیشگس نمیاد جای تو دلشوره دارم من
[01:11.000 --> 01:15.000]  بهت قول میدم صفت نیز
[01:15.000 --> 01:19.000]  لاغال برای تو راحت باش
[01:19.000 --> 01:24.000]  دورم از تو و دنیای تو راحت باش
[01:24.000 --> 01:31.000]  هیشگس نمیاد جای تو دلشوره دارم من
[01:31.000 --> 01:33.000]  منم از تو
[01:52.000 --> 01:57.000]  از عشق هرچیزی که میشناسم و از من گرفتی
[01:57.000 --> 02:02.000]  تو تو باقی منده احساسم و از من گرفتی
[02:02.000 --> 02:05.000]  و میخوای من باشی و یادت بره
[02:05.000 --> 02:09.000]  مایی وجو

In [2]:
# translate to english
!whisper MohsenYeganeh.mp3 --language Persian --task translate  --model large-v2 \
                           --temperature 0.01 --output_dir translate --best_of 3

[00:00.000 --> 00:03.000]  PART 13
[00:30.000 --> 00:34.000]  You want to leave, for her sake
[00:34.000 --> 00:38.000]  But your heart is burning, you imagine
[00:38.000 --> 00:42.000]  You are my lover, don't be afraid of this game every day
[00:42.000 --> 00:46.000]  People are all gone, heart is burning
[00:46.000 --> 00:50.000]  Think about these heartburns for two days
[00:51.000 --> 00:55.000]  I promise you, it's not hard
[00:55.000 --> 00:59.000]  At least be comfortable for you
[00:59.000 --> 01:03.000]  Stay away from you and your world
[01:03.000 --> 01:10.000]  No one will replace you, I have a heartburn
[01:11.000 --> 01:15.000]  I promise you, it's not hard
[01:15.000 --> 01:19.000]  At least be comfortable for you
[01:19.000 --> 01:24.000]  Stay away from you and your world
[01:24.000 --> 01:30.000]  No one will replace you, I have a heartburn
[01:30.000 --> 01:34.000]  I promise you, it's not hard
[01:52.000 --> 01:56.000]  You took everything I know from love
[01:56.0

In [3]:
!whisper --help

usage: whisper [-h]
               [--model {tiny.en,tiny,base.en,base,small.en,small,medium.en,medium,large-v1,large-v2,large}]
               [--model_dir MODEL_DIR] [--device DEVICE]
               [--output_dir OUTPUT_DIR]
               [--output_format {txt,vtt,srt,tsv,json,all}]
               [--verbose VERBOSE] [--task {transcribe,translate}]
               [--language {af,am,ar,as,az,ba,be,bg,bn,bo,br,bs,ca,cs,cy,da,de,el,en,es,et,eu,fa,fi,fo,fr,gl,gu,ha,haw,he,hi,hr,ht,hu,hy,id,is,it,ja,jw,ka,kk,km,kn,ko,la,lb,ln,lo,lt,lv,mg,mi,mk,ml,mn,mr,ms,mt,my,ne,nl,nn,no,oc,pa,pl,ps,pt,ro,ru,sa,sd,si,sk,sl,sn,so,sq,sr,su,sv,sw,ta,te,tg,th,tk,tl,tr,tt,uk,ur,uz,vi,yi,yo,zh,Afrikaans,Albanian,Amharic,Arabic,Armenian,Assamese,Azerbaijani,Bashkir,Basque,Belarusian,Bengali,Bosnian,Breton,Bulgarian,Burmese,Castilian,Catalan,Chinese,Croatian,Czech,Danish,Dutch,English,Estonian,Faroese,Finnish,Flemish,French,Galician,Georgian,German,Greek,Gujarati,Haitian,Haitian Creole,Hausa,Hawaiian,Hebrew,Hi